The file can be downloaded here:
https://www.data.gouv.fr/fr/datasets/fichier-des-equides/
or directly :
https://www.data.gouv.fr/s/resources/fichier-des-equides/20141201-185229/Equides.csv

Le fichier de 200 Mo contient autours de 3 millions de lignes

In [6]:
import csv
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from anonymizer.anonymity import (get_k, get_anonymities,
                                  less_anonym_groups)
from anonymizer.diversity import (get_l,
                                  get_diversities,
                                  diversity_distribution,
                                  less_diverse_groups
                                )
from anonymizer.transformations import (first_letters,
                                       last_letters,
                                       local_aggregation)
from anonymizer.transformations import str_drop
from anonymizer.anonymDF import AnonymDataFrame

from anonymizer.config_anonymizer import config
import os
import io

In [ ]:
path_data = config['PATH']['DATA']
equides = pd.read_csv(path_data, sep = ";", encoding = "ISO-8859-1", header=None, low_memory = False)

CParserError: Error tokenizing data. C error: out of memory

In [3]:
nom_de_colonnes = ['Race',
                   'Sexe',
                   'Robe',
                   'Date de naissance',
                   'Pays de naissance',
                   'Nom',
                   'Destiné à la consommation humaine',
                   'Date de mort']
equides.columns = nom_de_colonnes
equides.head()

,Race,Sexe,Robe,Date de naissance,Pays de naissance,Nom,Destiné à la consommation humaine,Date de mort
0,OR. INCONNUE,H,BAI,NaN,NaN,HOMME,NaN,
1,OR. INCONNUE,H,BAI,NaN,NaN,GITAN,NaN,
2,PONEY,H,ISABELLE,NaN,NaN,SCOOTER,NaN,
3,ARABE-BARBE,M,GRIS ROUANNE,NaN,NaN,VLADISLAS,NaN,
4,OR. INCONNUE,M,PIE BAI,NaN,NaN,SYLVER,NaN,


In [4]:
variables_supprimees = ['Date de mort']
equides = equides.drop(variables_supprimees,1)

In [5]:
equides['Date de naissance'] = last_letters(equides['Date de naissance'],6)
equides = equides.fillna('non renseigné')
equides = equides.applymap(lambda x: x.strip())
equides.replace('', 'non renseigné', inplace=True)

In [6]:
equides['Race'] = equides['Race'].str.lower()
liste_races = equides['Race'].unique().tolist()
len(liste_races)

179

In [7]:
for word in ['inconnu', 'anglo-arabe', 'welsh', 'aa compl.']:
    for race in liste_races :
        if word in race:
            print(word, race)
            equides['Race'] = equides['Race'].replace(race, word)

liste_races = equides['Race'].unique().tolist()
len(liste_races)

inconnu or. inconnue
inconnu race inconnue
anglo-arabe *anglo-arabe*
anglo-arabe anglo-arabe
welsh welsh cob
welsh welsh pony
welsh welsh type cob
welsh welsh mountain
welsh welsh
welsh welsh section k
welsh welsh section x
aa compl. *aa compl.*


171

In [8]:
ordre_aggregation = ['Race',
                     'Sexe',
                     'Robe',
                     'Pays de naissance',
                     'Destiné à la consommation humaine',
                     'Date de naissance']

In [10]:
kanonym_equides = local_aggregation(equides.copy(), 5, ordre_aggregation[:-1])

In [ ]:
kanonym_equides = local_aggregation(kanonym_equides, 5, [ordre_aggregation[-1]], method = "year")

In [13]:
kanonym_equides

,Race,Sexe,Robe,Date de naissance,Pays de naissance,Nom,Destiné à la consommation humaine
0,inconnu,H,BAI,non renseigné,non renseigné,HOMME,non renseigné
1,inconnu,H,BAI,non renseigné,non renseigné,GITAN,non renseigné
2,poney,F ou M ou H,GRIS ou GRIS ROUANNE ou BAI BRUN ou BAI ZAIN o...,non renseigné,FRANCE ou BELGIQUE ou non renseigné,SCOOTER,non renseigné
3,arabe-barbe,F ou M,GRIS ou BAI ou GRIS ROUANNE ou BAI FONCE ou BA...,non renseigné,TUNISIE ou non renseigné,VLADISLAS,non renseigné
4,inconnu,M,PIE BAI,non renseigné,non renseigné,SYLVER,non renseigné
5,inconnu,M,BAI,non renseigné,non renseigné,NIAGARA,N ou non renseigné
6,inconnu,H,BAI BRUN FONCE,non renseigné,non renseigné,RUIZ DE BOYSE,non renseigné
7,inconnu,H,PIE ALEZAN,non renseigné,non renseigné,NOUGAT DE BOYSE,N ou non renseigné
8,inconnu,H,PIE ALEZAN,non renseigné,non renseigné,MAC DE BOYSE,N ou non renseigné
9,kon. warmb. ned,M ou H,BAI FONCE,1987,PAYS-BAS,FELIX,non renseigné


In [11]:
len(equides)

3104041